In [1]:
from urllib.request import urlopen
import pandas as pd
import folium
import json
import numpy as np
import branca.colormap as cm
def polygon_area(xs, ys):
    """https://en.wikipedia.org/wiki/Centroid#Of_a_polygon"""
    # https://stackoverflow.com/a/30408825/7128154
    return 0.5 * (np.dot(xs, np.roll(ys, 1)) - np.dot(ys, np.roll(xs, 1)))

def polygon_centroid(xs, ys):
    """https://en.wikipedia.org/wiki/Centroid#Of_a_polygon"""
    xy = np.array([xs, ys])
    c = np.dot(xy + np.roll(xy, 1, axis=1),
               xs * np.roll(ys, 1) - np.roll(xs, 1) * ys
               ) / (6 * polygon_area(xs, ys))
    return c

path_geojson='https://rezultatevot-map-files.s3.eu-central-1.amazonaws.com/'

#counties json
with urlopen(path_geojson+'counties.geojson') as response:
    counties = json.load(response)
    
y_map, x_map=46,24.98682108
mymap = folium.Map(location=[y_map, x_map], zoom_start=6.5, tiles="Stamen Terrain")
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)

cen=0
area=0
tooltip = "Click me!"
for c in counties['features']:
    ys=[c['geometry']['coordinates'][0][-i-1][0] for i in range(len(c['geometry']['coordinates'][0]))]
    xs=[c['geometry']['coordinates'][0][-i-1][1] for i in range(len(c['geometry']['coordinates'][0]))]
    a=polygon_area(xs, ys)
    po=polygon_centroid(xs, ys)
    cen+=po*a
    
    tooltip = c['properties']['name']
    popup='https://www.google.com/maps/place/'+str(po[0])+','+str(po[1])
    folium.Marker([po[0],po[1]], popup=popup, tooltip=tooltip).add_to(mymap)

    area+=a
po=(cen/area)
tooltip = 'Romania'
popup='https://www.google.com/maps/place/'+str(po[0])+','+str(po[1])
folium.Marker([po[0],po[1]], popup=popup, tooltip=tooltip).add_to(mymap)

style_function = lambda x: {'fillColor': '#D2D2FF' if
                            x['properties']['name']=='Olt' else
                            '#FFD2D2'}

folium.GeoJson(counties, name="geojson",style_function=style_function).add_to(mymap)

mymap.save('rocenterm.html')
mymap